In [1]:
# Alpha_vantage_API_KEY= "4KI6MT403YQOW06H"
def getStockData(ticker, timePeriod = "compact" "full",Alpha_vantage_API_KEY= "4KI6MT403YQOW06H" ):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={ticker}&apikey={Alpha_vantage_API_KEY}&&outputsize={timePeriod}&datatype=json&interval=5min&time_period=60"
    return url

In [2]:
import requests

In [3]:
res = requests.get(getStockData("AAPL","full"))

stockData = res.json()

In [4]:
stockData["Time Series (Daily)"]

{'2023-06-02': {'1. open': '181.03',
  '2. high': '181.78',
  '3. low': '179.26',
  '4. close': '180.95',
  '5. adjusted close': '180.95',
  '6. volume': '61996913',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2023-06-01': {'1. open': '177.7',
  '2. high': '180.12',
  '3. low': '176.9306',
  '4. close': '180.09',
  '5. adjusted close': '180.09',
  '6. volume': '68901809',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2023-05-31': {'1. open': '177.325',
  '2. high': '179.35',
  '3. low': '176.76',
  '4. close': '177.25',
  '5. adjusted close': '177.25',
  '6. volume': '99313268',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2023-05-30': {'1. open': '176.96',
  '2. high': '178.99',
  '3. low': '176.57',
  '4. close': '177.3',
  '5. adjusted close': '177.3',
  '6. volume': '55964401',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2023-05-26': {'1. open': '173.32',
  '2. high': '175.77',
  '3

In [3]:
import pandas as pd

def json_to_dataframe(json, dataLength =0.8):
    df = pd.DataFrame.from_dict(json, orient='index')
    df.index.name = 'date'
    df = df.rename(columns={'date': 'date',"1. open": "open" ,"2. high":"high","3. low":"low",
                                        "4. close":"close","5. adjusted close":"adjusted close","6. volume":"volume",
                                        "7. dividend amount":"dividend amount","8. split coefficient":"split coefficient"})
    df = df.iloc[::-1]
    df.index =pd.to_datetime(df.index)



    percentage = dataLength
    indx = int(len(df) * percentage)
    df = df[indx:]
    return df

In [4]:
import os

def save_df_to_csv(df, ticker,file_name,folder_path="E:\Semesters\Fyp prepation\Forcasted_Data\Simple_Forcast\/"):
    # Check if the folder exists and create it if it doesn't
    if not os.path.exists(f"{folder_path}{ticker}"):
        os.makedirs(f"{folder_path}{ticker}")

    # Save the DataFrame to a CSV file in the specified folder
    df.to_csv(os.path.join(f"{folder_path}{ticker}", f"{ticker}_{file_name}.csv"))

In [5]:
from GetTimeSeriesPredictions import simpleTimeSeriesPred_LSTM

def getSaveSimpleTimeSeriesPred_LSTM(ticker,dataframe =None, loop_back=7, number_of_dense_layers=2,timePeriod = "compact" "full", data_Length =0.79,save_results = False):
    
    if dataframe == None:
        try:
            print("Fetching the stock Data from Alpha Vintage...")
            res = requests.get(getStockData(ticker,timePeriod))
            stockData = res.json()
            stockData= stockData["Time Series (Daily)"]
            dataframe = json_to_dataframe(stockData, data_Length)
            print("Fetching Sucessfull! ")
        except:
            print("Failed to Fetch Data from Alpha Vintage")
            raise     
        
        
    try:
        print("Getting predictions")
        predections, train_pred, test_pred, model = simpleTimeSeriesPred_LSTM(dataframe,loop_back, number_of_dense_layers)
        print("Got the Predictions Sucessfully!")
    except:
        print("Failed to get Predictions")
        raise
    
    if save_results!= False:
        try:
            save_df_to_csv(predections,ticker,"predections")
            save_df_to_csv(train_pred,ticker,"train_pred")
            save_df_to_csv(test_pred,ticker,"test_pred")
            print("Results Saved Sucesfully! ")
        except:
            print("Failed to Save the Results")
    
    return predections, train_pred, test_pred, model

    
    

In [6]:
getSaveSimpleTimeSeriesPred_LSTM("MSFT",timePeriod="full", save_results=True)

Fetching the stock Data from Alpha Vintage...
Fetching Sucessfull! 
Getting predictions
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 200)               166400    
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 2)                 402       
                                                                 
Total params: 166,802
Trainable params: 166,802
Non-trainable params: 0
_________________________________________________________________
None
12/12 [==============================] - 1s 3ms/step
Train Mean Absolute Error: 181.07852
Train Root Mean Squared Error: 193.32016
Test Mean Absolute Error: 274.84528
Test Root Mean Squared Error: 276.29095
Got the Predic

(             pred+0
 date               
 2023-06-02 330.0455
 2023-06-05 329.6014,
              pred+0   pred+1
 date                        
 2018-07-02  93.3091  92.0298
 2018-07-03  93.1216  91.8388
 2018-07-05  93.3747  92.1303
 2018-07-06  93.3049  92.0226
 2018-07-09  93.5578  92.2862
 ...             ...      ...
 2021-11-29 341.6760 341.0111
 2021-11-30 338.8303 338.0474
 2021-12-01 338.6707 338.0644
 2021-12-02 337.1590 336.5089
 2021-12-03 335.3701 334.7521
 
 [864 rows x 2 columns],
              pred+0   pred+1
 date                        
 2021-12-16 337.0748 336.3487
 2021-12-17 337.2714 336.6098
 2021-12-20 334.6447 333.8040
 2021-12-21 331.7010 330.9528
 2021-12-22 329.0627 328.3557
 ...             ...      ...
 2023-05-26 319.1701 318.5026
 2023-05-30 322.2642 321.8115
 2023-05-31 326.8328 326.5733
 2023-06-01 329.1569 328.8986
 2023-06-02 330.0455 329.6014
 
 [367 rows x 2 columns],
 <keras.engine.sequential.Sequential at 0x16b050f28b0>)

In [7]:
def getSaveSimpleTimeSeriesPred_LSTM(ticker,dataframe =None, loop_back=7, number_of_dense_layers=2,timePeriod = "compact" "full", data_Length =0.79,save_results = False, folder_path="E:\Semesters\Fyp prepation\Forcasted_Data\Simple_Forcast\/",special_deco_fileName=""):
    
    if dataframe == None:
        try:
            print("Fetching the stock Data from Alpha Vintage...")
            res = requests.get(getStockData(ticker,timePeriod))
            stockData = res.json()
            stockData= stockData["Time Series (Daily)"]
            dataframe = json_to_dataframe(stockData, data_Length)
            print("Fetching Sucessfull! ")
        except:
            print("Failed to Fetch Data from Alpha Vintage")
            raise     
        
        
    try:
        print("Getting predictions")
        predections, train_pred, test_pred, model = simpleTimeSeriesPred_LSTM(dataframe,loop_back, number_of_dense_layers)
        print("Got the Predictions Sucessfully!")
    except:
        print("Failed to Get Predictions")
        raise
    
    if save_results!= False:
        try:
            save_df_to_csv(predections,ticker,f"predections_{special_deco_fileName}", folder_path)
            save_df_to_csv(train_pred,ticker,f"train_pred_{special_deco_fileName}", folder_path)
            save_df_to_csv(test_pred,ticker,f"test_pred_{special_deco_fileName}", folder_path)
            print("Results Saved Sucesfully! ")
        except:
            print("Failed to Save the Results")
    
    return predections, train_pred, test_pred, model


In [9]:
getSaveSimpleTimeSeriesPred_LSTM("MSFT",loop_back=7, number_of_dense_layers=2, timePeriod="full", save_results=True)

Fetching the stock Data from Alpha Vintage...
Fetching Sucessfull! 
Getting predictions
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 200)               166400    
                                                                 
 dropout_2 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 2)                 402       
                                                                 
Total params: 166,802
Trainable params: 166,802
Non-trainable params: 0
_________________________________________________________________
None
12/12 [==============================] - 1s 3ms/step
Train Mean Absolute Error: 176.11635
Train Root Mean Squared Error: 188.59148
Test Mean Absolute Error: 269.62424
Test Root Mean Squared Error: 271.19806
Got the Pred

(             pred+0
 date               
 2023-06-02 328.2999
 2023-06-05 329.0286,
              pred+0   pred+1
 date                        
 2018-07-02  89.6571  88.4207
 2018-07-03  89.6262  88.3231
 2018-07-05  90.0116  88.7418
 2018-07-06  90.0749  88.6889
 2018-07-09  90.2870  88.9466
 ...             ...      ...
 2021-11-29 338.8993 340.0767
 2021-11-30 335.2197 336.3126
 2021-12-01 335.3083 336.6998
 2021-12-02 333.5248 334.7016
 2021-12-03 331.6646 333.0063
 
 [864 rows x 2 columns],
              pred+0   pred+1
 date                        
 2021-12-16 334.5011 335.3132
 2021-12-17 334.3432 335.4435
 2021-12-20 330.5535 331.5365
 2021-12-21 327.3061 328.5384
 2021-12-22 324.2518 325.3895
 ...             ...      ...
 2023-05-26 315.3893 316.1224
 2023-05-30 319.2135 320.1805
 2023-05-31 324.6347 325.7407
 2023-06-01 327.6472 328.5576
 2023-06-02 328.2999 329.0286
 
 [367 rows x 2 columns],
 <keras.engine.sequential.Sequential at 0x16b345ede20>)

In [141]:
import pandas as pd
import datetime

# Function to generate the next date excluding weekends
def generate_next_date(current_date):
    next_date = current_date + datetime.timedelta(days=1)
    while next_date.weekday() >= 5:  # Skip Saturday (5) and Sunday (6)
        next_date += datetime.timedelta(days=1)
    return next_date
def convert_columns_to_rows(df):
    # Convert index to datetime if needed
    df.index = pd.to_datetime(df.index)


    # Get the current date from the DataFrame index
    current_date = df.index[0]

    # Create an empty DataFrame to store the converted data
    new_df = pd.DataFrame(columns=['date', 'pred+0'])

    # Iterate over each column and convert values into rows
    for column in df.columns:
        current_value = df[column][0]

        # Generate the next date excluding weekends
        next_date = generate_next_date(current_date)

        # Add the current date and value as a row in the new DataFrame
        new_row = pd.DataFrame({'date': [current_date], 'pred+0': [current_value]})
        new_df = pd.concat([new_df, new_row], ignore_index=True)

        # Update the current date for the next iteration
        current_date = next_date

    new_df.set_index('date', inplace=True)
    return new_df
